#2025年度バージョン


## 前準備


gensimのバージョンアップといったん再起動が必要


In [ ]:
!pip install --upgrade gensim

import os
os.kill(os.getpid(), 9)

形態素解析Mecabの設定


In [ ]:
!apt-get install mecab mecab-ipadic-utf8
!pip install mecab-python3
!ln -s  /etc/mecabrc /usr/local/etc/mecabrc

青空文庫から著作権の切れたテキストを持ってくる

In [ ]:
import requests

urls = ['https://www.aozora.gr.jp/cards/000148/files/752_14964.html',   # 坊ちゃん
        'https://www.aozora.gr.jp/cards/000148/files/773_14560.html',   # こころ
        'https://www.aozora.gr.jp/cards/000081/files/456_15050.html',   # 銀河鉄道の夜
        'https://www.aozora.gr.jp/cards/000081/files/46605_31178.html', # やまなし
        'https://www.aozora.gr.jp/cards/000879/files/127_15260.html',   # 羅生門
        'https://www.aozora.gr.jp/cards/000879/files/92_14545.html',    # 蜘蛛の糸
        'https://www.aozora.gr.jp/cards/000035/files/1567_14913.html',  # 走れメロス
        'https://www.aozora.gr.jp/cards/000296/files/47061_29420.html', # 学問のすすめ
       ]

htmls = []
for url in urls:
  response = requests.get(url)
  print(response.encoding) # SJISのはずなのにISO-8859-1
  response.encoding = response.apparent_encoding # encoding情報の修正
  print(response.encoding) # S_JIS
  htmlfile = response.text # response.encodingでdecodeしてunicodeの文字列にする
  print(htmlfile)
  htmls.append(htmlfile)


タグなど不要なものを削除する

In [ ]:
from bs4 import BeautifulSoup
import lxml


table = str.maketrans({'\r':'','\n':'','\u3000':'', ' ':'', '\t':''}) #改行と空白文字消去のためのテーブル

sentences = []

for htmlfile in htmls:
  soup = BeautifulSoup(htmlfile, "lxml") # html/xmlタグを消す
  for rubi in soup.findAll(["rt", "rp"]): # このタグの中身はルビなのでいらない
    rubi.decompose()
  for metadata in  soup.findAll("div", attrs={ "class": ["bibliographical_information", "notation_notes"] }):
    metadata.decompose()
  for metadata in  soup.findAll("div", attrs={ "id": "card" }):
    metadata.decompose()
  string = soup.body.get_text().strip()
  string = string.translate(table)
  sentences.append(string)

## BoWと各種トピックベクトルの生成

In [ ]:
import gensim
import MeCab
import seaborn

tagger = MeCab.Tagger("-Owakati")
# 各sentence（ドキュメント）に対して分かち書き
tokenized_docs = [tagger.parse(sentence).split() for sentence in sentences]

# 単語のリスト（辞書）をつくる。IDと語のmapping
dic = gensim.corpora.Dictionary(tokenized_docs)

# 分かち書きされた各ドキュメントをBoWベクトル（頻度）にする
BOW_vecs = [dic.doc2bow(tokenized_doc) for tokenized_doc in tokenized_docs]

# lsiを行い、ドキュメントベクトル（topic分布ベクトル）を求める
lsimodel = gensim.models.LsiModel(BOW_vecs, id2word=dic, num_topics=5)
lsi_vecs = [lsimodel[BOW_vec] for BOW_vec in BOW_vecs]

# nmfを行い、ドキュメントベクトル（topic分布ベクトル）を求める
nmfmodel = gensim.models.Nmf(BOW_vecs, id2word=dic, num_topics=5)
nmf_vecs = [nmfmodel[BOW_vec] for BOW_vec in BOW_vecs]

# ldaを行い、ドキュメントベクトル（topic分布ベクトル）を求める
ldamodel = gensim.models.LdaModel(BOW_vecs, id2word=dic, num_topics=5, passes=10)
lda_vecs = [ldamodel[BOW_vec] for BOW_vec in BOW_vecs]


# 各ドキュメントとの類似度を求めるための準備
index = gensim.similarities.MatrixSimilarity(BOW_vecs, num_features=len(dic))
# 各ドキュメント(BOW_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
bowsimilarities = [index[BOW_vec] for BOW_vec in BOW_vecs]


# 各ドキュメントとの類似度を求めるための準備
lsiindex = gensim.similarities.MatrixSimilarity(lsi_vecs, num_features=len(dic))
# 各ドキュメント(lsi_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
lsisimilarities = [lsiindex[lsi_vec] for lsi_vec in lsi_vecs]

# 各ドキュメントとの類似度を求めるための準備
nmfindex = gensim.similarities.MatrixSimilarity(nmf_vecs, num_features=len(dic))
# 各ドキュメント(nmf_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
nmfsimilarities = [nmfindex[nmf_vec] for nmf_vec in nmf_vecs]

# 各ドキュメントとの類似度を求めるための準備
ldaindex = gensim.similarities.MatrixSimilarity(lda_vecs, num_features=len(dic))
# 各ドキュメント(lda_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
ldasimilarities = [ldaindex[lda_vec] for lda_vec in lda_vecs]


ベクトル間の類似度の確認

In [ ]:
seaborn.heatmap(bowsimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(lsisimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(nmfsimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(ldasimilarities, annot=True, cmap='Blues')

## BoWをTFIDFにして各種トピックベクトルの生成

In [ ]:
import gensim
import MeCab
import seaborn

tagger = MeCab.Tagger("-Owakati")
# 各sentence（ドキュメント）に対して分かち書き
tokenized_docs = [tagger.parse(sentence).split() for sentence in sentences]

# 単語のリスト（辞書）をつくる。IDと語のmapping
dic = gensim.corpora.Dictionary(tokenized_docs)

# 分かち書きされた各ドキュメントをBoWベクトル（頻度）にする
BOW_vecs = [dic.doc2bow(tokenized_doc) for tokenized_doc in tokenized_docs]

# tfidfを計算し、BoWベクトル（tfidf）を求める
tfidfmodel = gensim.models.TfidfModel(BOW_vecs)
tfidf_vecs = [tfidfmodel[BOW_vec] for BOW_vec in BOW_vecs]

# lsiを行い、ドキュメントベクトル（topic分布ベクトル）を求める
lsimodel = gensim.models.LsiModel(tfidf_vecs, id2word=dic, num_topics=5)
lsi_vecs = [lsimodel[tfidf_vec] for tfidf_vec in tfidf_vecs]

# nmfを行い、ドキュメントベクトル（topic分布ベクトル）を求める
nmfmodel = gensim.models.Nmf(tfidf_vecs, id2word=dic, num_topics=5)
nmf_vecs = [nmfmodel[tfidf_vec] for tfidf_vec in tfidf_vecs]

# ldaを行い、ドキュメントベクトル（topic分布ベクトル）を求める
ldamodel = gensim.models.LdaModel(tfidf_vecs, id2word=dic, num_topics=5, passes=10)
lda_vecs = [ldamodel[tfidf_vec] for tfidf_vec in tfidf_vecs]


# 各ドキュメントとの類似度を求めるための準備
tfidfindex = gensim.similarities.MatrixSimilarity(tfidf_vecs, num_features=len(dic))
# 各ドキュメント(tfidf_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
tfidfsimilarities = [tfidfindex[tfidf_vec] for tfidf_vec in tfidf_vecs]

# 各ドキュメントとの類似度を求めるための準備
lsiindex = gensim.similarities.MatrixSimilarity(lsi_vecs, num_features=len(dic))
# 各ドキュメント(lsi_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
lsisimilarities = [lsiindex[lsi_vec] for lsi_vec in lsi_vecs]

# 各ドキュメントとの類似度を求めるための準備
nmfindex = gensim.similarities.MatrixSimilarity(nmf_vecs, num_features=len(dic))
# 各ドキュメント(nmf_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
nmfsimilarities = [nmfindex[nmf_vec] for nmf_vec in nmf_vecs]

# 各ドキュメントとの類似度を求めるための準備
ldaindex = gensim.similarities.MatrixSimilarity(lda_vecs, num_features=len(dic))
# 各ドキュメント(lda_vec)と、準備したindexの各ドキュメントとのコサイン類似度をとる
ldasimilarities = [ldaindex[lda_vec] for lda_vec in lda_vecs]

ベクトル間の類似度の確認

In [ ]:
seaborn.heatmap(tfidfsimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(lsisimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(nmfsimilarities, annot=True, cmap='Blues')

In [ ]:
seaborn.heatmap(ldasimilarities, annot=True, cmap='Blues')